# BiLSTM

In [ ]:
# Red LSTM bidireccional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM

# model
model = keras.models.Sequential()
model.add(keras.Input(shape=(5754,96)))
#model.add(layers.Masking(mask_value=0., input_shape=(5754, 96)))
model.add(layers.Bidirectional(LSTM(512, return_sequences=True, activation='tanh')))
model.add(layers.Dropout(0.2))
model.add(layers.Bidirectional(LSTM(128, return_sequences=True, activation='tanh')))
model.add(layers.Dropout(0.2))
model.add(layers.Bidirectional(LSTM(64, return_sequences=True, activation='tanh')))
model.add(layers.Dense(3, activation = 'softmax'))
print(model.summary())

early_stopping = EarlyStopping(
    monitor='val_accuracy', # Requiere de un conjunto de validación
    patience=200,         
    verbose=1,           
    restore_best_weights=True 
)

model_checkpoint = ModelCheckpoint(
    'mejor_modelo_sin_run_mas_metricas.h5',   # Nombre del archivo donde se guardarán los pesos
    monitor='val_accuracy', 
    save_best_only=True,
    verbose=1      
)

loss = keras.losses.CategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy", 'Precision', 'Recall']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

batch_size = 16
epochs = 200

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[model_checkpoint], validation_split=0.2)

model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=2)

# LSTM CON 3 CAPAS DENSAS

In [ ]:
# RED LSTM con 3 capas densas
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM

# model
model = keras.models.Sequential()
model.add(keras.Input(shape=(5754,96)))
#model.add(layers.Masking(mask_value=0., input_shape=(5754, 96)))
model.add(LSTM(512, return_sequences=True, activation='tanh'))
model.add(layers.Dropout(0.2))
model.add(LSTM(512, return_sequences=True, activation='tanh'))
model.add(LSTM(512, return_sequences=True, activation='tanh'))
model.add(layers.Dropout(0.2))
model.add(LSTM(512, return_sequences=True, activation='tanh'))
model.add(layers.Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(layers.Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(layers.Dropout(0.2))
model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(layers.Dense(32, activation='softmax'))
model.add(layers.Dense(16, activation='softmax'))
model.add(layers.Dense(3, activation = 'softmax'))
print(model.summary())

early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    patience=200,          
    verbose=1,            
    restore_best_weights=True  
)

model_checkpoint = ModelCheckpoint(
    'mejor_modelo_sin_run_mas_densas.h5',   
    monitor='val_accuracy', 
    save_best_only=True, 
    verbose=1            
)

loss = keras.losses.CategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy", 'Precision', 'Recall']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

batch_size = 16
epochs = 200

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[model_checkpoint], validation_split=0.2)

model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=2)

# CNN 1D

In [ ]:
# RED CNN1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, AveragePooling1D, BatchNormalization, UpSampling1D, TimeDistributed, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()

#model.add(layers.Masking(mask_value=0., input_shape=(5754, 96)))
model.add(Conv1D(32, 3, strides=1, activation='relu', padding='same', input_shape=(5754, 96)))
model.add(Conv1D(32, 3, strides=1, activation='relu', padding='same'))
model.add(AveragePooling1D(pool_size=2, strides=2))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(Conv1D(64, 3, strides=1, activation='relu', padding='same'))
model.add(Conv1D(64, 3, strides=1, activation='relu', padding='same'))
model.add(AveragePooling1D(pool_size=2, strides=2))
model.add(Dropout(0.2))

model.add(BatchNormalization())
model.add(Conv1D(128, 3, strides=1, activation='relu', padding='same'))
model.add(Conv1D(128, 3, strides=1, activation='relu', padding='same'))
model.add(AveragePooling1D(pool_size=2, strides=2))
model.add(Dropout(0.2)) 

model.add(BatchNormalization())
model.add(Conv1D(256, 3, strides=1, activation='relu', padding='same'))
model.add(Conv1D(256, 3, strides=1, activation='relu', padding='same'))
model.add(AveragePooling1D(pool_size=2, strides=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])

model_checkpoint = ModelCheckpoint('mejor_modelo_cnn1d.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.summary()

history = model.fit(X_train, Y_train, validation_split=0.2, epochs=2000, batch_size=16, verbose=2, callbacks=[model_checkpoint])

model.evaluate(X_test, Y_test, batch_size=16)

# CNN 2D

In [ ]:
# Cambio de señal a imagen
import numpy as np

num_frames_per_image = 14
new_width = 42
new_height = 32

num_images_per_sequence = X_train_.shape[1] // num_frames_per_image

# Reshape de X_train para agrupar cada 10 frames
X_reshaped = X_train.reshape(X_train_.shape[0], num_images_per_sequence, num_frames_per_image * X_train_.shape[2])
X_reshaped = X_reshaped.reshape(-1, new_width, new_height, 1)  # -1 calcula automáticamente el tamaño necesario

# Ajusta las etiquetas Y_train para que coincidan con el nuevo agrupamiento
Y_reshaped = Y_train_[:, ::num_frames_per_image, :].reshape(-1, 3)

print(X_reshaped.shape)
print(Y_reshaped.shape)

In [ ]:
# RED CNN2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Dropout, AveragePooling2D, BatchNormalization, Flatten, Dense, TimeDistributed

model = Sequential([
    Conv2D(32, kernel_size=3, strides=(1, 1), activation='relu', padding='same', input_shape=(42, 32, 1)),
    Dropout(0.2),
    AveragePooling2D(pool_size=2, strides=2, padding='same'),

    BatchNormalization(),
    Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    Dropout(0.2),
    AveragePooling2D(pool_size=2, padding='same'),

    BatchNormalization(),
    Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    Dropout(0.2),
    AveragePooling2D(pool_size=2, padding='same'),

    Conv2D(256, kernel_size=3, activation='relu', padding='same'),
    AveragePooling2D(pool_size=2, padding='same'),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])

model_checkpoint = ModelCheckpoint('mejor_modelo.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.summary()

history = model.fit(X_reshaped, Y_reshaped, validation_split=0.2, epochs=200, batch_size=16, verbose=2, callbacks=[model_checkpoint])

model.evaluate(X_test, Y_test, batch_size=16)

# BiRNN

In [ ]:
# Red recurrente bidireccional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import SimpleRNN

# model
model = keras.models.Sequential()
model.add(keras.Input(shape=(5754,96)))
#model.add(layers.Masking(mask_value=0., input_shape=(5754, 96)))
model.add(layers.Bidirectional(SimpleRNN(512, return_sequences=True, activation='tanh')))
model.add(layers.Dropout(0.2))
model.add(layers.Bidirectional(SimpleRNN(128, return_sequences=True, activation='tanh')))
model.add(layers.Dropout(0.2))
model.add(layers.Bidirectional(SimpleRNN(64, return_sequences=True, activation='tanh')))
model.add(layers.Dense(3, activation = 'softmax'))
print(model.summary())

early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    patience=200,     
    verbose=1,        
    restore_best_weights=True
)  

model_checkpoint = ModelCheckpoint(
    'mejor_modelo_sin_run_mas_metricas.h5',  
    monitor='val_accuracy',  
    save_best_only=True, 
    verbose=1           
)

loss = keras.losses.CategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy", 'Precision', 'Recall']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

batch_size = 16
epochs = 200

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=2, callbacks=[model_checkpoint], validation_split=0.2)

model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=2)